In [70]:
import datetime
import pymongo
import os
from pymongo import MongoClient
from dotenv import load_dotenv
ruta_actual = os.getcwd() 
ruta_padre = os.path.dirname(ruta_actual)
load_dotenv('venv/.env')
import datetime
import random
import json
from bson.objectid import ObjectId

In [71]:
a = os.getenv('connection_url_SanManuel')
a

In [72]:
connection_url_SanManuel =  'mongodb://eefACE:9UzYm7sB5HxR4WqE3TnGpJ2MvDfXcZvA6L8O1K0I7U3N5Y8T4R6W9Q3E6T2O8H7D3E6S4L9I7T8O1L0O@192.168.100.11:27018/' 
ecommerce_main = 'Ecommerce'
eCommerce_inventario = [connection_url_SanManuel, ecommerce_main, 'inventario']
eCommerce_usuarios = [connection_url_SanManuel, ecommerce_main, 'usuario']

In [73]:
def coneccionDB(mongo_uri:str,database_name:str,collection_name:str):
    '''Permite realizar una coneccion a una colecccion en base de datos'''
    
    try:
        client = MongoClient(mongo_uri)
        db = client[database_name]
        collection = db[collection_name]
        print(f'coneccion exitosa a la coleccion: {database_name}.{collection_name}')
    except Exception as e:
        print(e)
    return collection

## Asignar un precio a los productos

In [74]:
def generar_precio_unitario():
    return random.randint(30, 200)
update = False
# Actualiza todos los documentos en la colección (nota, todos los prodcutos tienen el mismo precio)
if update:
    result = coneccionDB(*eCommerce_inventario).update_many({}, {'$set': {'precioUnitario': generar_precio_unitario()}})


## ejemplo de un usuario

In [75]:
usuario = coneccionDB(*eCommerce_usuarios).find_one({'nombre': "Daniel García"})
usuario

coneccion exitosa a la coleccion: Ecommerce.usuario


{'_id': ObjectId('65c63fbca5315cbe1489b148'),
 'usuario': 'Al92202453',
 'nombre': 'Daniel García',
 'denominacion': 'SwineSustenance Solutions Inc.',
 'password': '$2b$12$GOHmVEj0X5Wz8SglY.RvNeNVaYuCWMUuqKvx/Lut5AXIqCIpy1snK',
 'email': 'swine.sus.s@outlook.com',
 'proveedor': 1,
 'telefono': '2211847999',
 'celular': '2211847999',
 'picture': '/images/imagenes/user.png'}

## ejemplo de una  lista aleatoria de  productos

In [76]:
pipeline = [{'$sample': {'size': 3}}]
productos  = list(coneccionDB(*eCommerce_inventario).aggregate(pipeline))
productos

coneccion exitosa a la coleccion: Ecommerce.inventario


[{'_id': ObjectId('6671b1b37be252ce7e978325'),
  'codigoBarras': '',
  'fecha': datetime.datetime(2024, 3, 7, 0, 0),
  'nombreOriginal': 'RECHARGEABLE G-SPOT DUAL STIMULATOR with rolling ball',
  'nombre': 'ESTIMULADOR DUAL DEL PUNTO G con bola rodante',
  'cajas': 1,
  'cantidadCaja': 50,
  'Total': 50,
  'precioUnitario': 103,
  'foto': '/img/estimulador_dual_punto_g_rosado_morado.jpg'},
 {'_id': ObjectId('6671b1b37be252ce7e978310'),
  'codigoBarras': '',
  'fecha': datetime.datetime(2024, 3, 7, 0, 0),
  'nombreOriginal': '10X DUAL TEASER with vibration & licking funtions',
  'nombre': '10X DOBLE AVANCE',
  'cajas': 1,
  'cantidadCaja': 50,
  'Total': 50,
  'precioUnitario': 90,
  'foto': '/img/control_remoto_doble_avance.jpg'},
 {'_id': ObjectId('6671b1b37be252ce7e978351'),
  'codigoBarras': '',
  'fecha': datetime.datetime(2024, 3, 7, 0, 0),
  'nombreOriginal': 'LUXURY FETISH Ball Gag',
  'nombre': 'Mordaza de bola',
  'cajas': 1,
  'cantidadCaja': 50,
  'Total': 50,
  'precioUnita

In [77]:
def simularCompra(productos:list):

    for producto in productos:
        cantidad = random.randint(a=1,b=10)
        subtotal = cantidad*producto['precioUnitario']

        producto['cantidad'] = cantidad
        producto['subtotal'] = subtotal
    
    return productos
        

In [78]:
# firstkeys = ['codigoBarras', 'nombre', 'precioUnitario']
# productos = list(map(lambda d: {k: d[k] for k in firstkeys}, productos))
# productos = simularCompra(productos=productos)
# productos

In [79]:
# totalCompra    =  sum([producto['subtotal'] for producto in productos])
# totalArticulos =  sum([producto['cantidad'] for producto in productos])

## crear un registro de un ticket


In [80]:
def crearTicket():
    registro = dict()

    usuario = coneccionDB(*eCommerce_usuarios).find_one({'nombre': "Daniel García"})
    registro['idUser'] = usuario['usuario']
    registro['userName'] = usuario['nombre']
    #
    sale_size = random.randint(a=1, b= 10)
    pipeline = [{'$sample': {'size': sale_size}}]
    productos  = list(coneccionDB(*eCommerce_inventario).aggregate(pipeline))

    firstkeys = ['codigoBarras', 'nombre', 'precioUnitario']
    productos = list(map(lambda d: {k: d[k] for k in firstkeys}, productos))
    productos = simularCompra(productos=productos)
    
    registro['compras'] = productos
    #
    registro['totalCompra']    =  sum([producto['subtotal'] for producto in productos])
    registro['totalArticulos'] =  sum([producto['cantidad'] for producto in productos])
    registro['metidoPago']     =  random.choice(['Efectivo', 'PayPal', 'Tarjeta'])

    return registro

In [86]:
ticket = crearTicket()
productos = ticket

coneccion exitosa a la coleccion: Ecommerce.usuario
coneccion exitosa a la coleccion: Ecommerce.inventario


In [87]:
eCommerceInventarioDB = coneccionDB(*eCommerce_inventario)
for producto in productos:
        
        producto_bd = list(eCommerceInventarioDB.find({"nombre":{"$in":[producto['nombre']]}}))
        if len(producto_bd) > 0:
            
            try:
                cantidad_inventario = producto_bd[0]['Total']
                nuevo_total = cantidad_inventario - producto['cantidad']

                if nuevo_total < 0:
                    print("No existe una cantidad en almacen para organizar esa demanda")
                elif nuevo_total == 0:
                    print("ya no queda mas en inventario")
                else:
                    print("Ahora solo quedan {} unidades del producto {}".format(nuevo_total, producto['nombre']))
        
            except:
                None

coneccion exitosa a la coleccion: Ecommerce.inventario


TypeError: string indices must be integers

In [83]:
def EvaluarCapacidad():
    productos  = list(coneccionDB(*eCommerce_inventario).find({"codigoBarras":{"$in":["240116SSB04"]}}))
    return productos

In [84]:
EvaluarCapacidad()

coneccion exitosa a la coleccion: Ecommerce.inventario


[]

##  Funcion para obtener un producto por nombre

In [85]:
def getProduct(product_name:str):
    """ 
    obtener la informacion 
    """
    eCommerce_inventario = [connection_url_SanManuel, ecommerce_main, 'inventario']
    product = coneccionDB(*eCommerce_inventario).find_one({'nombre': product_name})

    return product

